# GRAPH WORLD: SBM Generator

In [5]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import collections
import enum
import math
import random
from typing import Dict, Sequence, List, Tuple
from sklearn.preprocessing import normalize

from graph_tool.all import *
import dataclasses
import numpy as np

class MatchType(enum.Enum):
  """Indicates type of feature/graph membership matching to do.
    RANDOM: feature memberships are generated randomly.
    NESTED: for # feature groups >= # graph groups. Each feature cluster is a
      sub-cluster of a graph cluster. Multiplicity of sub-clusters per
      graph cluster is kept as uniform as possible.
    GROUPED: for # feature groups <= # graph groups. Each graph cluster is a
      sub-cluster of a feature cluster. Multiplicity of sub-clusters per
      feature cluster is kept as uniform as possible.
  """
  RANDOM = 1
  NESTED = 2
  GROUPED = 3
@dataclasses.dataclass
class StochasticBlockModel:
  """Stores data for stochastic block model graphs with features.
  Attributes:
    graph: graph-tool Graph object.
    graph_memberships: list of integer node classes.
    node_features: numpy array of node features.
    feature_memberships: list of integer node feature classes.
    edge_features: map from edge tuple to numpy array. Only stores undirected
      edges, i.e. (0, 1) will be in the map, but (1, 0) will not be.
  """
  graph: graph_tool.Graph = Ellipsis
  graph_memberships: np.ndarray = Ellipsis
  node_features: np.ndarray = Ellipsis
  feature_memberships: np.ndarray = Ellipsis
  edge_features: Dict[Tuple[int, int], np.ndarray] = Ellipsis


def _GetNestingMap(large_k, small_k):
  """Given two group sizes, computes a "nesting map" between groups.
  This function will produce a bipartite map between two sets of "group nodes"
  that will be used downstream to partition nodes in a bigger graph. The map
  encodes which groups from the larger set are nested in certain groups from
  the smaller set.
  As currently implemented, nesting is assigned as evenly as possible. If
  large_k is an integer multiple of small_k, each smaller-set group will be
  mapped to exactly (large_k/small_k) larger-set groups. If there is a
  remainder r, the first r smaller-set groups will each have one extra nested
  larger-set group.
  Args:
    large_k: (int) size of the larger group set
    small_k: (int) size of the smaller group set
  Returns:
    nesting_map: (dict) map from larger group set indices to lists of
      smaller group set indices
  """
  min_multiplicity = int(math.floor(large_k / small_k))
  max_bloated_group_index = large_k - small_k * min_multiplicity - 1
  nesting_map = collections.defaultdict(list)
  pos = 0
  for i in range(small_k):
    for _ in range(min_multiplicity + int(i <= max_bloated_group_index)):
      nesting_map[i].append(pos)
      pos += 1
  return nesting_map


def _GenerateFeatureMemberships(
    graph_memberships,
    num_groups=None,
    match_type=MatchType.RANDOM):
  """Generates a feature membership assignment.
  Args:
    graph_memberships: (list) the integer memberships for the graph SBM
    num_groups: (int) number of groups. If None, defaults to number of unique
      values in graph_memberships.
    match_type: (MatchType) see the enum class description.
  Returns:
    memberships: a int list - index i contains feature group of node i.
  """
  # Parameter checks
  if num_groups is not None and num_groups == 0:
    raise ValueError("argument num_groups must be None or positive")
  graph_num_groups = len(set(graph_memberships))
  if num_groups is None:
    num_groups = graph_num_groups

  # Compute memberships
  memberships = []
  if match_type == MatchType.GROUPED:
    if num_groups > graph_num_groups:
      raise ValueError(
        "for match type GROUPED, must have num_groups <= graph_num_groups")
    nesting_map = _GetNestingMap(graph_num_groups, num_groups)
    # Creates deterministic map from (smaller) graph clusters to (larger)
    # feature clusters.
    reverse_nesting_map = {}
    for feature_cluster, graph_cluster_list in nesting_map.items():
      for cluster in graph_cluster_list:
        reverse_nesting_map[cluster] = feature_cluster
    for cluster in graph_memberships:
      memberships.append(reverse_nesting_map[cluster])
  elif match_type == MatchType.NESTED:
    if num_groups < graph_num_groups:
      raise ValueError(
        "for match type NESTED, must have num_groups >= graph_num_groups")
    nesting_map = _GetNestingMap(num_groups, graph_num_groups)
    # Creates deterministic map from (smaller) feature clusters to (larger)
    # graph clusters.
    for graph_cluster_id, feature_cluster_ids in nesting_map.items():
      sorted_feature_cluster_ids = sorted(feature_cluster_ids)
      num_feature_groups = len(sorted_feature_cluster_ids)
      feature_pi = np.ones(num_feature_groups) / num_feature_groups
      num_graph_cluster_nodes = np.sum(
        [i == graph_cluster_id for i in graph_memberships])
      sub_memberships = _GenerateNodeMemberships(num_graph_cluster_nodes,
                                                 feature_pi)
      sub_memberships = [sorted_feature_cluster_ids[i] for i in sub_memberships]
      memberships.extend(sub_memberships)
  else:  # MatchType.RANDOM
    memberships = random.choices(range(num_groups), k=len(graph_memberships))
  return np.array(sorted(memberships))


def _ComputeExpectedEdgeCounts(num_edges, num_vertices,
                               pi,
                               prop_mat):
  """Computes expected edge counts within and between communities.
  Args:
    num_edges: expected number of edges in the graph.
    num_vertices: number of nodes in the graph
    pi: interable of non-zero community size proportions. Must sum to 1.0, but
      this check is left to the caller of this internal function.
    prop_mat: square, symmetric matrix of community edge count rates. Entries
      must be non-negative, but this check is left to the caller.
  Returns:
    symmetric matrix with shape prop_mat.shape giving expected edge counts.
  """
  scale = np.matmul(pi, np.matmul(prop_mat, pi)) * num_vertices ** 2
  prob_mat = prop_mat * num_edges / scale
  return np.outer(pi, pi) * prob_mat * num_vertices ** 2


def _ComputeCommunitySizes(num_vertices, pi):
  """Helper function of GenerateNodeMemberships to compute group sizes.
  Args:
    num_vertices: number of nodes in graph.
    pi: interable of non-zero community size proportions.
  Returns:
    community_sizes: np vector of group sizes. If num_vertices * pi[i] is a
      whole number (up to machine precision), community_sizes[i] will be that
      number. Otherwise, this function accounts for rounding errors by making
      group sizes as balanced as possible (i.e. increasing smallest groups by
      1 or decreasing largest groups by 1 if needed).
  """
  community_sizes = [int(x * num_vertices) for x in pi]
  if sum(community_sizes) != num_vertices:
    size_order = np.argsort(community_sizes)
    delta = sum(community_sizes) - num_vertices
    adjustment = np.sign(delta)
    if adjustment == 1:
      size_order = np.flip(size_order)
    for i in range(int(abs(delta))):
      community_sizes[size_order[i]] -= adjustment
  return community_sizes


def _GenerateNodeMemberships(num_vertices, pi):
  """Gets node memberships for sbm.
  Args:
    num_vertices: number of nodes in graph.
    pi: interable of non-zero community size proportions. Must sum to 1.0, but
      this check is left to the caller of this internal function.
  Returns:
    np vector of ints representing community indices.
  """
  community_sizes = _ComputeCommunitySizes(num_vertices, pi)
  memberships = np.zeros(num_vertices, dtype=int)
  node = 0
  for i in range(len(pi)):
    memberships[range(node, node + community_sizes[i])] = i
    node += community_sizes[i]
  return memberships


def SimulateSbm(sbm_data,
                num_vertices,
                num_edges,
                pi,
                prop_mat,
                out_degs=None):
  """Generates a stochastic block model, storing data in sbm_data.graph.
  This function uses networkx.generate_sbm. Refer to that
  documentation for more information on the model and parameters.
  Args:
    sbm_data: StochasticBlockModel dataclass to store result data.
    num_vertices: (int) number of nodes in the graph.
    num_edges: (int) expected number of edges in the graph.
    pi: iterable of non-zero community size proportions. Must sum to 1.0.
    prop_mat: square, symmetric matrix of community edge count rates.
    out_degs: Out-degree propensity for each node. If not provided, a constant
      value will be used. Note that the values will be normalized inside each
      group, if they are not already so.
  Returns: (none)
  """
  if round(abs(np.sum(pi) - 1.0), 12) != 0:
    raise ValueError("entries of pi ( must sum to 1.0")
  if prop_mat.shape[0] != len(pi) or prop_mat.shape[1] != len(pi):
    raise ValueError("prop_mat must be k x k where k = len(pi)")
  sbm_data.graph_memberships = _GenerateNodeMemberships(num_vertices, pi)
  edge_counts = _ComputeExpectedEdgeCounts(num_edges, num_vertices, pi, prop_mat)
  sbm_data.graph = graph_tool.generation.generate_sbm(sbm_data.graph_memberships, edge_counts, out_degs)
  graph_tool.generation.remove_self_loops(sbm_data.graph)
  graph_tool.generation.remove_parallel_edges(sbm_data.graph)
  sbm_data.graph.reindex_edges()


def SimulateFeatures(sbm_data,
                     center_var,
                     feature_dim,
                     num_groups,
                     match_type=MatchType.RANDOM,
                     cluster_var=1.0,
                     normalize_features=True):
  """Generates node features using multivate normal mixture model.
  This function does nothing and throws a warning if
  sbm_data.graph_memberships is empty. Run SimulateSbm to fill that field.
  Feature data is stored as an attribute of sbm_data named 'node_features'.
  Args:
    sbm_data: StochasticBlockModel dataclass to store result data.
    center_var: (float) variance of feature cluster centers. When this is 0.0,
      the signal-to-noise ratio is 0.0. When equal to cluster_var, SNR is 1.0.
    feature_dim: (int) dimension of the multivariate normal.
   num_groups: (int) number of centers. Generated by a multivariate normal with
     mean zero and covariance matrix cluster_var * I_{feature_dim}.
    match_type: (MatchType) see sbm_simulator.MatchType for details.
    cluster_var: (float) variance of feature clusters around their centers.
  Raises:
    RuntimeWarning: if simulator has no graph or a graph with no nodes.
  """
  if sbm_data.graph_memberships is None:
    raise RuntimeWarning("No graph_memberships found: no features generated. "
                         "Run SimulateSbm to generate graph_memberships.")

  # Get memberships
  sbm_data.feature_memberships = _GenerateFeatureMemberships(
    graph_memberships=sbm_data.graph_memberships,
    num_groups=num_groups,
    match_type=match_type)

  # Get centers
  centers = []
  center_cov = np.identity(feature_dim) * center_var
  cluster_cov = np.identity(feature_dim) * cluster_var
  for _ in range(num_groups):
    center = np.random.multivariate_normal(
      np.zeros(feature_dim), center_cov, 1)[0]
    centers.append(center)
  features = []
  for cluster_index in sbm_data.feature_memberships:
    feature = np.random.multivariate_normal(centers[cluster_index], cluster_cov,
                                            1)[0]
    features.append(feature)
  features = np.array(features)
  if normalize_features:
    features = normalize(features)
  sbm_data.node_features = features


def SimulateEdgeFeatures(sbm_data,
                         feature_dim,
                         center_distance=0.0,
                         cluster_variance=1.0):
  """Generates edge feature distribution via inter-class vs intra-class.
  Edge feature data is stored as an sbm_data attribute named `edge_feature`, a
  dict from 2-tuples of node IDs to numpy vectors.
  Edge features have two centers: one at (0, 0, ....) and one at
  (center_distance, center_distance, ....) for inter-class and intra-class
  edges (respectively). They are generated from a multivariate normal with
  covariance matrix = cluster_variance * I_d.
  Requires non-None `graph` and `graph_memberships` attributes in sbm_data.
  Use SimulateSbm to generate them. Throws warning if either are None.
  Args:
    sbm_data: StochasticBlockModel dataclass to store result data.
    feature_dim: (int) dimension of the multivariate normal.
    center_distance: (float) per-dimension distance between the intra-class and
      inter-class means. Increasing this makes the edge feature signal stronger.
    cluster_variance: (float) variance of clusters around their centers.
  Raises:
    RuntimeWarning: if simulator has no graph or a graph with no nodes.
  """
  if sbm_data.graph is None:
    raise RuntimeWarning("SbmSimulator has no graph: no features generated.")
  if sbm_data.graph.num_vertices() == 0:
    raise RuntimeWarning("graph has no nodes: no features generated.")
  if sbm_data.graph_memberships is None:
    raise RuntimeWarning("graph has no memberships: no features generated.")

  center0 = np.zeros(shape=(feature_dim,))
  center1 = np.ones(shape=(feature_dim,)) * center_distance
  covariance = np.identity(feature_dim) * cluster_variance
  sbm_data.edge_features = {}
  for edge in sbm_data.graph.edges():
    vertex1 = int(edge.source())
    vertex2 = int(edge.target())
    edge_tuple = tuple(sorted((vertex1, vertex2)))
    if (sbm_data.graph_memberships[vertex1] ==
        sbm_data.graph_memberships[vertex2]):
      center = center1
    else:
      center = center0
    sbm_data.edge_features[edge_tuple] = np.random.multivariate_normal(
      center, covariance, 1)[0]


def GenerateStochasticBlockModelWithFeatures(
    num_vertices,
    num_edges,
    pi,
    prop_mat,
    out_degs=None,
    feature_center_distance=0.0,
    feature_dim=0,
    num_feature_groups=1,
    feature_group_match_type=MatchType.RANDOM,
    feature_cluster_variance=1.0,
    edge_feature_dim=0,
    edge_center_distance=0.0,
    edge_cluster_variance=1.0,
    normalize_features=True):
  """Generates stochastic block model (SBM) with node features.
  Args:
    num_vertices: number of nodes in the graph.
    num_edges: expected number of edges in the graph.
    pi: interable of non-zero community size proportions. Must sum to 1.0.
    prop_mat: square, symmetric matrix of community edge count rates. Example:
      if diagonals are 2.0 and off-diagonals are 1.0, within-community edges are
      twices as likely as between-community edges.
    out_degs: Out-degree propensity for each node. If not provided, a constant
      value will be used. Note that the values will be normalized inside each
      group, if they are not already so.
    feature_center_distance: distance between feature cluster centers. When this
      is 0.0, the signal-to-noise ratio is 0.0. When equal to
      feature_cluster_variance, SNR is 1.0.
    feature_dim: dimension of node features.
    num_feature_groups: number of feature clusters.
    feature_group_match_type: see sbm_simulator.MatchType.
    feature_cluster_variance: variance of feature clusters around their centers.
      centers. Increasing this weakens node feature signal.
    edge_feature_dim: dimension of edge features.
    edge_center_distance: per-dimension distance between the intra-class and
      inter-class means. Increasing this strengthens the edge feature signal.
    edge_cluster_variance: variance of edge clusters around their centers.
      Increasing this weakens the edge feature signal.
  Returns:
    result: a StochasticBlockModel data class.
  """
  result = StochasticBlockModel()
  SimulateSbm(result, num_vertices, num_edges, pi, prop_mat, out_degs)
  SimulateFeatures(result, feature_center_distance,
                   feature_dim,
                   num_feature_groups,
                   feature_group_match_type,
                   feature_cluster_variance,
                   normalize_features)
  SimulateEdgeFeatures(result, edge_feature_dim,
                       edge_center_distance,
                       edge_cluster_variance)
  return result


# Helper function to create the "Pi" vector for the SBM model (the
# ${num_communities}-simplex vector giving relative community sizes) from
# the `community_size_slope` config field. See the config proto for details.
def MakePi(num_communities: int, community_size_slope: float) -> np.ndarray:
  pi = np.array(range(num_communities)) * community_size_slope
  pi += np.ones(num_communities)
  pi /= np.sum(pi)
  return pi


# Helper function to create the "PropMat" matrix for the SBM model (square
# matrix giving inter-community Poisson means) from the config parameters,
# particularly `p_to_q_ratio`. See the config proto for details.
'''def MakePropMat(num_communities: int, p_to_q_ratio: float) -> np.ndarray:
  prop_mat = np.ones((num_communities, num_communities))
  np.fill_diagonal(prop_mat, p_to_q_ratio)
  return prop_mat'''
def MakePropMat(num_communities: int, p_to_q_ratio: float) -> np.ndarray:
  prop_mat = np.ones((num_communities, num_communities))
  np.fill_diagonal(prop_mat, p_to_q_ratio)
  return prop_mat


# Helper function to create a degree set that follows a power law for the
# 'out_degs' parameter in SBM construction.
def MakeDegrees(power_exponent, min_deg, num_vertices):
  degrees = np.zeros(num_vertices)
  k_min = min_deg
  k_max = num_vertices
  gamma = power_exponent
  for i in range(num_vertices):
      degrees[i] = int(power_law(k_min, k_max, np.random.uniform(0,1), gamma))
  return degrees

def convert_networkx(gt_graph):
    # create a new empty NetworkX graph
    nx_graph = nx.Graph()

    # add nodes to the NetworkX graph
    for node in gt_graph.vertices():
        nx_graph.add_node(int(node))
    # add edges to the NetworkX graph
    for edge in gt_graph.edges():
        src, dst = int(edge.source()), int(edge.target())
        nx_graph.add_edge(src, dst)
    nx_graph.remove_nodes_from(list(nx.isolates(nx_graph)))
    return nx_graph
def gt_to_nx(gt_graph, labels):
    nx_graph = nx.Graph()
    edge_list = [(int(e.source()), int(e.target())) for e in gt_graph.edges()]
    nx_graph.add_edges_from(edge_list)
    nx.set_node_attributes(nx_graph, {i: group for i, group in enumerate(labels)},
                           "group")
    return nx_graph

# Helper function of MakeDegrees to construct power law samples
def power_law(k_min, k_max, y, gamma):
  return ((k_max**(-gamma+1) - k_min**(-gamma+1))*y  + k_min**(-gamma+1.0))**(1.0/(-gamma + 1.0))

# Create Synthetic Dataset

In [1]:
import os
import json
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import math
from scipy.sparse.linalg import eigsh

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            raise TypeError(
                "Unserializable object {} of type {}".format(obj, type(obj))
            )
        return json.JSONEncoder.default(self, obj)
    
def get_from_json(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    return data

def molloy_reed(g):
  all_degree =   np.array(g.degree())[:,1]
  degs = np.delete(all_degree,-1)
  k = degs.mean()
  k2 = np.mean(degs ** 2)
  beta = k2/k
  return beta

def global_feature(g): 
    feature = {}
    M = len(g.edges())
    N = len(g)
    degs =   np.array(g.degree())[:,1]
    min_k = np.min(degs)
    k1 = degs.mean()
    k2 = np.mean(degs** 2)
    div = k2 - k1**2
    A = nx.to_scipy_sparse_array(g, weight='weight',dtype=float,format='csr')
    adj_lams = np.sort(eigsh(A, k=N-1, which='LA', return_eigenvectors=False))
    feature["nodes"] = N
    feature["edges"] = M
    feature["averagedegree"] = k1
    feature["heterogeneity"] = div/k1
    feature["density"] = (2*M)/(N*(N-1))
    feature["resilience"] = molloy_reed(g)
    #power_law_exponent = 1 + N / sum(np.log(degs/min_k))
    feature["modularity"] = nx_comm.modularity(g,nx_comm.label_propagation_communities(g))
    try:
        varepsilons = nx.algorithms.distance_measures.eccentricity(g).values()
        feature["eccentricity"]=  np.mean(list(varepsilons))
        feature["diameter"] = nx.algorithms.distance_measures.diameter(g)
        feature["radius"] = nx.algorithms.distance_measures.radius(g)
    except:
        lcc = max(nx.connected_components(g), key=len)
        subGraph = g.subgraph(lcc)
        varepsilons = nx.algorithms.distance_measures.eccentricity(subGraph).values()
        feature["eccentricity"]=  np.mean(list(varepsilons))
        feature["diameter"] = nx.algorithms.distance_measures.diameter(subGraph)
        feature["radius"] = nx.algorithms.distance_measures.radius(subGraph)
    feature["spectral_radius"]=adj_lams[-1]
    feature["spectral_gap"]=  adj_lams[-1]-adj_lams[-2]
    feature["natural_connectivity"]= np.log2(sum(np.exp(np.real(adj_lams)))/len(adj_lams))
    feature["global_efficiency"] = nx.algorithms.efficiency_measures.global_efficiency(g)
    feature["assortativity"] = nx.algorithms.assortativity.degree_assortativity_coefficient(g)
    #global_properties =np.hstack((nodes, edges,density,resilience,heterogeneity,power_law_exponent, modularity,eccentricity,diameter,radius,spectral_radius,spectral_gap,natural_connectivity,assortativity))
    return feature
def gen_regular_graph(n,d):
    # Generate a regular graph
    G = nx.Graph()

    # Add edges between nodes in a regular pattern
    for i in range(n):
        for j in range(1, d+1):
            G.add_edge(i, (i+j) % n)
    return G

def gen_sbm(n,p):
    N = n
    M = 3
    NUM_CLUSTERS = 5
    SLOPE = 0.1
    FEATURE_DIM = 16
    EDGE_DIM = 4
    INTER_LINK_STRENGTH = 0.15
    P2Q = 16.0
    POWER_EXPONENT = 2.0
    dataset = GenerateStochasticBlockModelWithFeatures(
            num_vertices=N,
            num_edges=N*5,
            pi=MakePi(NUM_CLUSTERS, SLOPE),
            #prop_mat=MakePropMat(NUM_CLUSTERS, P2Q),
            prop_mat=MakePropMat(NUM_CLUSTERS, p),
            out_degs=MakeDegrees(POWER_EXPONENT, M, N),
            feature_center_distance=1.0,
            feature_dim=FEATURE_DIM,
            num_feature_groups=1,
            feature_group_match_type=MatchType.GROUPED,
            feature_cluster_variance=1.0,
            edge_feature_dim=EDGE_DIM,
            edge_center_distance=0.0,
            edge_cluster_variance=1.0,
            normalize_features=True)
    G = gt_to_nx(dataset.graph, dataset.graph_memberships)
    return G

def gen_graph(cur_n,parameter, g_type):
    if g_type == 'erdos_renyi':
        g = nx.erdos_renyi_graph(n=cur_n, p=parameter)
    elif g_type == 'powerlaw':
        g = nx.powerlaw_cluster_graph(n=cur_n, m=3, p=0.05)
    elif g_type == 'small-world':
        g = nx.connected_watts_strogatz_graph(n=cur_n, k=parameter[0], p=parameter[1])
    elif g_type == 'barabasi_albert':
        g = nx.barabasi_albert_graph(n=cur_n, m=parameter)
    elif g_type == 'regular':
        g = gen_regular_graph(n=cur_n, d=parameter)
    elif g_type == 'sbm':
        g = gen_sbm(n=cur_n, p = parameter)
    lcc = max(nx.connected_components(g), key=len)
    Subgraph = g.subgraph(lcc)
    return Subgraph


def get_graph_degree(graph,feature,cur_n):
    noderange = []
    properties = []
    files= []
    for i in range(1,6):
        print(i, end=(", "))
        for graphtype in graph:
            print(graphtype, end=(", "))
            for parameter in graph[graphtype]:
                G= gen_graph(cur_n, parameter, graphtype)
                if graphtype == 'small-world':
                    filename = graphtype+"_"+str(cur_n)+"_"+str(parameter[0])+"_"+str(parameter[1])+"_"+str(i)
                else:
                    filename = graphtype+"_"+str(cur_n)+"_"+str(parameter)+"_"+str(i)
                files.append(filename)
                #plt.figure(figsize=(15, 20), dpi=120)
                #nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
                nx.write_edgelist(G, '../Dataset/SyntheticGraph/New/'+filename+'.txt',data=False)
                feature[filename] = global_feature(G)
                feature[filename]["parameter"] = parameter
    return feature
def get_graph_homogeneity(graph,feature):
    noderange = []
    properties = []
    files= []
    averagedegree = 4
    for i in range(1,5):
        print(i, end=(", "))
        for graphtype in graph:
            if graphtype == 'erdos_renyi':
                parameter = [0.1 for n in graph[graphtype]]
            elif graphtype =='small-world':
                parameter = [averagedegree for n in graph[graphtype]]
            elif graphtype =='barabasi_albert':
                parameter = [3 for n in graph[graphtype]]
            for k, cur_n in enumerate(graph[graphtype]):
                G= gen_graph(cur_n, parameter[k], graphtype)
                filename = graphtype+"_"+str(cur_n)+"_"+"{:.3f}".format(parameter[k])+"_"+str(i)
                files.append(filename)
                #plt.figure(figsize=(15, 20), dpi=120)
                #nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
                nx.write_edgelist(G, '../Dataset/SyntheticGraph/Homogeneity/New/'+filename+'.txt',data=False)
                feature[filename] = global_feature(G)
                feature[filename]["parameter"] = parameter
    return feature

## Average Degree: Different Average Degree of same size

In [ ]:
graph = {'erdos_renyi': [0.005,0.025,0.05,0.075,0.1],
         'barabasi_albert': [2,3,4,5,6],
         'small-world': [2,3,4,5,6],
        }
graph = {'erdos_renyi': [0.008, 0.016, 0.024, 0.032, 0.04, 0.048, 0.056, 0.064, 0.072, 0.08],
         'barabasi_albert': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
         #'small-world': [[i,0.1]for i in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]],
         'small-world': [[i,0.15]for i in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]],
        }
numNodes = 250
features = {}#get_from_json("../Dataset/featuresNew.json")
global_feature_degree = get_graph_degree(graph, {}, numNodes)
#feature = {"degree": global_feature_degree} 
features["synthetic"] =  global_feature_degree 
with open("../Dataset/featuresNew.json", "w") as outfile:
    json.dump(features, outfile, cls=NpEncoder)

In [7]:
graph = {'erdos_renyi': [0.005,0.025,0.05,0.075,0.1],
         'barabasi_albert': [2,3,4,5,6],
         'small-world': [2,3,4,5,6],
        }
graph = {'erdos_renyi': [0.001, 0.020, 0.030, 0.050, 0.0750, 0.100, 0.150, 0.200, 0.250,0.300],
         'small-world': [[i,0.15]for i in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]],
         'sbm': [i for i in [5,10,15,20,25,30,35,40,45,50]],
        }
numNodes = 250
features = get_from_json("../Dataset/featuresNew.json")
global_feature_degree = get_graph_degree(graph, features['synthetic'], numNodes)
#feature = {"degree": global_feature_degree} 
features["synthetic"] =  global_feature_degree 
with open("../Dataset/featuresNew.json", "w") as outfile:
    json.dump(features, outfile, cls=NpEncoder)

1, erdos_renyi, small-world, sbm, 2, erdos_renyi, small-world, sbm, 3, erdos_renyi, small-world, sbm, 4, erdos_renyi, small-world, sbm, 5, erdos_renyi, small-world, sbm, 

In [8]:
!tar -C ../Dataset/SyntheticGraph/New -czf New.tar .


## Edge homogeneity: Same Average Degrees of Different Size

In [49]:
graph = {'erdos_renyi': [50,100,200,350,500],
         'barabasi_albert':  [50,100,200,350,500],
         'small-world':  [50,100,200,350,500],
        }
graph = {'erdos_renyi': [50,100,150, 200,250,300,350,400,450,500],
         'barabasi_albert':  [50,100,150, 200,250,300,350,400,450,500],
         'small-world': [50,100,150, 200,250,300,350,400,450,500],
        }
numNodes = 500
features = get_from_json("../Dataset/featuresNew.json")
global_feature_homogeneity = get_graph_homogeneity(graph, {})
features["homogeneity"]=  global_feature_homogeneity
with open("../Dataset/featuresNew.json", "w") as outfile:
    json.dump(features, outfile, cls=NpEncoder)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 

In [52]:
!tar -C ../Dataset/SyntheticGraph/Degree/New -czf New_Degree.tar .
!tar -C ../Dataset/SyntheticGraph/Homogeneity/New -czf New_Homogeneity.tar . 

In [ ]:
def get_graph(graph_type,feature):
    noderange = [50,750]
    properties = []
    files= []
    for graphtype in graph_type:
        print(graphtype, end=", ")
        for nodesize in noderange:
            G= gen_graph(nodesize, graphtype)
            filename = graphtype+"_"+str(nodesize)
            files.append(filename)
            #plt.figure(figsize=(15, 20), dpi=120)
            #nx.draw(G,nodelist=G.nodes, font_color='white' ,node_size = 900, with_labels=True)
            nx.write_edgelist(G, '../Dataset/SyntheticGraph/'+filename+'.txt',data=False)
            feature[filename] = global_feature(G)
    print(files)
    return feature
feature = get_from_json("../Dataset/features.json")
feature = get_graph(['erdos_renyi'], feature)
with open("../Dataset/features.json", "w") as outfile:
    json.dump(feature, outfile, cls=NpEncoder)

## Generate Synthetic Graph: GNNExplainer

In [ ]:
%matplotlib inline
"""gengraph.py
   Generating and manipulaton the synthetic graphs needed for the paper's experiments.
"""

import os

from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.colors as colors
from scipy.sparse.linalg import eigsh
# Set matplotlib backend to file writing
#plt.switch_backend("agg")

import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np

"""synthetic_structsim.py
    Utilities for generating certain graph shapes.
"""
import math

import networkx as nx
import numpy as np

""" featgen.py
Node feature generators.
"""
import abc


class FeatureGen(metaclass=abc.ABCMeta):
    """Feature Generator base class."""
    @abc.abstractmethod
    def gen_node_features(self, G):
        pass


class ConstFeatureGen(FeatureGen):
    """Constant Feature class."""
    def __init__(self, val):
        self.val = val

    def gen_node_features(self, G):
        feat_dict = {i:{'feat': np.array(self.val, dtype=np.float32)} for i in G.nodes()}
        #print ('feat_dict[0]["feat"]:', feat_dict[0]['feat'].dtype)
        nx.set_node_attributes(G, feat_dict)
        #print ('G.nodes[0]["feat"]:', G.nodes[0]['feat'].dtype)


class GaussianFeatureGen(FeatureGen):
    """Gaussian Feature class."""
    def __init__(self, mu, sigma):
        self.mu = mu
        if sigma.ndim < 2:
            self.sigma = np.diag(sigma)
        else:
            self.sigma = sigma

    def gen_node_features(self, G):
        feat = np.random.multivariate_normal(self.mu, self.sigma, G.number_of_nodes())
        feat_dict = {
                i: {"feat": feat[i]} for i in range(feat.shape[0])
            }
        nx.set_node_attributes(G, feat_dict)


class GridFeatureGen(FeatureGen):
    """Grid Feature class."""
    def __init__(self, mu, sigma, com_choices):
        self.mu = mu                    # Mean
        self.sigma = sigma              # Variance
        self.com_choices = com_choices  # List of possible community labels

    def gen_node_features(self, G):
        # Generate community assignment
        community_dict = {
            n: self.com_choices[0] if G.degree(n) < 4 else self.com_choices[1]
            for n in G.nodes()
        }

        # Generate random variable
        s = np.random.normal(self.mu, self.sigma, G.number_of_nodes())

        # Generate features
        feat_dict = {
            n: {"feat": np.asarray([community_dict[n], s[i]])}
            for i, n in enumerate(G.nodes())
        }

        nx.set_node_attributes(G, feat_dict)
        return community_dict
    
    
# Following GraphWave's representation of structural similarity


def clique(start, nb_nodes, nb_to_remove=0, role_start=0):
    """ Defines a clique (complete graph on nb_nodes nodes,
    with nb_to_remove  edges that will have to be removed),
    index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    nb_nodes    :    int correspondingraph to the nb of nodes in the clique
    role_start  :    starting index for the roles
    nb_to_remove:    int-- numb of edges to remove (unif at RDM)
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    a = np.ones((nb_nodes, nb_nodes))
    np.fill_diagonal(a, 0)
    graph = nx.from_numpy_matrix(a)
    edge_list = graph.edges().keys()
    roles = [role_start] * nb_nodes
    if nb_to_remove > 0:
        lst = np.random.choice(len(edge_list), nb_to_remove, replace=False)
        to_delete = [edge_list[e] for e in lst]
        graph.remove_edges_from(to_delete)
        for e in lst:
            roles[edge_list[e][0]] += 1
            roles[edge_list[e][1]] += 1
    mapping_graph = {k: (k + start) for k in range(nb_nodes)}
    graph = nx.relabel_nodes(graph, mapping_graph)
    return graph, roles


def cycle(start, len_cycle, role_start=0):
    """Builds a cycle graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + len_cycle))
    for i in range(len_cycle - 1):
        graph.add_edges_from([(start + i, start + i + 1)])
    graph.add_edges_from([(start + len_cycle - 1, start)])
    roles = [role_start] * len_cycle
    return graph, roles


def diamond(start, role_start=0):
    """Builds a diamond graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + 6))
    graph.add_edges_from(
        [
            (start, start + 1),
            (start + 1, start + 2),
            (start + 2, start + 3),
            (start + 3, start),
        ]
    )
    graph.add_edges_from(
        [
            (start + 4, start),
            (start + 4, start + 1),
            (start + 4, start + 2),
            (start + 4, start + 3),
        ]
    )
    graph.add_edges_from(
        [
            (start + 5, start),
            (start + 5, start + 1),
            (start + 5, start + 2),
            (start + 5, start + 3),
        ]
    )
    roles = [role_start] * 6
    return graph, roles


def tree(start, height, r=2, role_start=0):
    """Builds a balanced r-tree of height h
    INPUT:
    -------------
    start       :    starting index for the shape
    height      :    int height of the tree 
    r           :    int number of branches per node 
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a tree shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at role_start)
    """
    graph = nx.balanced_tree(r, height)
    roles = [0] * graph.number_of_nodes()
    return graph, roles


def fan(start, nb_branches, role_start=0):
    """Builds a fan-like graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    nb_branches :    int correspondingraph to the nb of fan branches
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph, roles = star(start, nb_branches, role_start=role_start)
    for k in range(1, nb_branches - 1):
        roles[k] += 1
        roles[k + 1] += 1
        graph.add_edges_from([(start + k, start + k + 1)])
    return graph, roles


def ba(start, width, role_start=0, m=5):
    """Builds a BA preferential attachment graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    width       :    int size of the graph
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.barabasi_albert_graph(width, m)
    graph.add_nodes_from(range(start, start + width))
    nids = sorted(graph)
    mapping = {nid: start + i for i, nid in enumerate(nids)}
    graph = nx.relabel_nodes(graph, mapping)
    roles = [role_start for i in range(width)]
    return graph, roles


def house(start, role_start=0):
    """Builds a house-like  graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + 5))
    graph.add_edges_from(
        [
            (start, start + 1),
            (start + 1, start + 2),
            (start + 2, start + 3),
            (start + 3, start),
        ]
    )
    # graph.add_edges_from([(start, start + 2), (start + 1, start + 3)])
    graph.add_edges_from([(start + 4, start), (start + 4, start + 1)])
    roles = [role_start, role_start, role_start + 1, role_start + 1, role_start + 2]
    return graph, roles


def grid(start, dim=2, role_start=0):
    """ Builds a 2by2 grid
    """
    grid_G = nx.grid_graph([dim, dim])
    grid_G = nx.convert_node_labels_to_integers(grid_G, first_label=start)
    roles = [role_start for i in grid_G.nodes()]
    return grid_G, roles


def star(start, nb_branches, role_start=0):
    """Builds a star graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    nb_branches :    int correspondingraph to the nb of star branches
    start       :    starting index for the shape
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + nb_branches + 1))
    for k in range(1, nb_branches + 1):
        graph.add_edges_from([(start, start + k)])
    roles = [role_start + 1] * (nb_branches + 1)
    roles[0] = role_start
    return graph, roles


def path(start, width, role_start=0):
    """Builds a path graph, with index of nodes starting at start
    and role_ids at role_start
    INPUT:
    -------------
    start       :    starting index for the shape
    width       :    int length of the path
    role_start  :    starting index for the roles
    OUTPUT:
    -------------
    graph       :    a house shape graph, with ids beginning at start
    roles       :    list of the roles of the nodes (indexed starting at
                     role_start)
    """
    graph = nx.Graph()
    graph.add_nodes_from(range(start, start + width))
    for i in range(width - 1):
        graph.add_edges_from([(start + i, start + i + 1)])
    roles = [role_start] * width
    roles[0] = role_start + 1
    roles[-1] = role_start + 1
    return graph, roles


def build_graph(
    width_basis,
    basis_type,
    list_shapes,
    start=0,
    rdm_basis_plugins=False,
    add_random_edges=0,
    m=5,
):
    """This function creates a basis (scale-free, path, or cycle)
    and attaches elements of the type in the list randomly along the basis.
    Possibility to add random edges afterwards.
    INPUT:
    --------------------------------------------------------------------------------------
    width_basis      :      width (in terms of number of nodes) of the basis
    basis_type       :      (torus, string, or cycle)
    shapes           :      list of shape list (1st arg: type of shape,
                            next args:args for building the shape,
                            except for the start)
    start            :      initial nb for the first node
    rdm_basis_plugins:      boolean. Should the shapes be randomly placed
                            along the basis (True) or regularly (False)?
    add_random_edges :      nb of edges to randomly add on the structure
    m                :      number of edges to attach to existing node (for BA graph)
    OUTPUT:
    --------------------------------------------------------------------------------------
    basis            :      a nx graph with the particular shape
    role_ids         :      labels for each role
    plugins          :      node ids with the attached shapes
    """
    if basis_type == "ba":
        
        basis, role_id = eval(basis_type)(start, width_basis, m=m)
    else:
        basis, role_id = eval(basis_type)(start, width_basis)
    n_basis, n_shapes = nx.number_of_nodes(basis), len(list_shapes)
    start += n_basis  # indicator of the id of the next node

    # Sample (with replacement) where to attach the new motifs
    if rdm_basis_plugins is True:
        plugins = np.random.choice(n_basis, n_shapes, replace=False)
    else:
        spacing = math.floor(n_basis / n_shapes)
        plugins = [int(k * spacing) for k in range(n_shapes)]
    seen_shapes = {"basis": [0, n_basis]}

    for shape_id, shape in enumerate(list_shapes):
        shape_type = shape[0]
        args = [start]
        if len(shape) > 1:
            args += shape[1:]
        args += [0]
        graph_s, roles_graph_s = eval(shape_type)(*args)
        n_s = nx.number_of_nodes(graph_s)
        try:
            col_start = seen_shapes[shape_type][0]
        except:
            col_start = np.max(role_id) + 1
            seen_shapes[shape_type] = [col_start, n_s]
        # Attach the shape to the basis
        basis.add_nodes_from(graph_s.nodes())
        basis.add_edges_from(graph_s.edges())
        basis.add_edges_from([(start, plugins[shape_id])])
        if shape_type == "cycle":
            if np.random.random() > 0.5:
                a = np.random.randint(1, 4)
                b = np.random.randint(1, 4)
                basis.add_edges_from([(a + start, b + plugins[shape_id])])
        temp_labels = [r + col_start for r in roles_graph_s]
        # temp_labels[0] += 100 * seen_shapes[shape_type][0]
        role_id += temp_labels
        start += n_s

    if add_random_edges > 0:
        # add random edges between nodes:
        for p in range(add_random_edges):
            src, dest = np.random.choice(nx.number_of_nodes(basis), 2, replace=False)
            basis.add_edges_from([(src, dest)])

    return basis, role_id, plugins

####################################
#
# Experiment utilities
#
####################################
def perturb(graph_list, p):
    """ Perturb the list of (sparse) graphs by adding/removing edges.
    Args:
        p: proportion of added edges based on current number of edges.
    Returns:
        A list of graphs that are perturbed from the original graphs.
    """
    perturbed_graph_list = []
    for G_original in graph_list:
        G = G_original.copy()
        edge_count = int(G.number_of_edges() * p)
        # randomly add the edges between a pair of nodes without an edge.
        for _ in range(edge_count):
            while True:
                u = np.random.randint(0, G.number_of_nodes())
                v = np.random.randint(0, G.number_of_nodes())
                if (not G.has_edge(u, v)) and (u != v):
                    break
            G.add_edge(u, v)
        perturbed_graph_list.append(G)
    return perturbed_graph_list


def join_graph(G1, G2, n_pert_edges):
    """ Join two graphs along matching nodes, then perturb the resulting graph.
    Args:
        G1, G2: Networkx graphs to be joined.
        n_pert_edges: number of perturbed edges.
    Returns:
        A new graph, result of merging and perturbing G1 and G2.
    """
    assert n_pert_edges > 0
    F = nx.compose(G1, G2)
    edge_cnt = 0
    while edge_cnt < n_pert_edges:
        node_1 = np.random.choice(G1.nodes())
        node_2 = np.random.choice(G2.nodes())
        F.add_edge(node_1, node_2)
        edge_cnt += 1
    return F


def preprocess_input_graph(G, labels, normalize_adj=False):
    """ Load an existing graph to be converted for the experiments.
    Args:
        G: Networkx graph to be loaded.
        labels: Associated node labels.
        normalize_adj: Should the method return a normalized adjacency matrix.
    Returns:
        A dictionary containing adjacency, node features and labels
    """
    adj = np.array(nx.to_numpy_matrix(G))
    if normalize_adj:
        sqrt_deg = np.diag(1.0 / np.sqrt(np.sum(adj, axis=0, dtype=float).squeeze()))
        adj = np.matmul(np.matmul(sqrt_deg, adj), sqrt_deg)

    existing_node = list(G.nodes)[-1]
    feat_dim = G.nodes[existing_node]["feat"].shape[0]
    f = np.zeros((G.number_of_nodes(), feat_dim), dtype=float)
    for i, u in enumerate(G.nodes()):
        f[i, :] = G.nodes[u]["feat"]

    # add batch dim
    adj = np.expand_dims(adj, axis=0)
    f = np.expand_dims(f, axis=0)
    labels = np.expand_dims(labels, axis=0)
    return {"adj": adj, "feat": f, "labels": labels}


####################################
#
# Generating synthetic graphs
#
###################################
def bahouse(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["house"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_house"
    return G, role_id, name

def bafan(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["fan"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_fan"
    return G, role_id, name

def baclique(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["clique"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_clique"
    return G, role_id, name

def bastar(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["star"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_star"
    return G, role_id, name

def badiamond(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["diamond"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_diamond"
    return G, role_id, name


def bacycle(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #1:
    Start with Barabasi-Albert graph and attach house-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  A list with length equal to number of nodes in the entire graph (basis
                          :  + shapes). role_id[i] is the ID of the role of node i. It is the label.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["cycle"]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes) + "_cycle"
    return G, role_id, name


def bagrid(nb_shapes=100, width_basis=60, feature_generator=None, m=5):
    """ Synthetic Graph #3:
    Start with Barabasi-Albert graph and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'grid') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here 'Barabasi-Albert' random graph).
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  number of edges to attach to existing node (for BA graph)
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph.
        name              :  A graph identifier
    """
    basis_type = "ba"
    list_shapes = [["grid", 3]] * nb_shapes

    plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0, m=5
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_grid"
    return G, role_id, name

def treecycle(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #4:
    Start with a tree and attach cycle-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'Tree').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["cycle", 6]] * nb_shapes

    fig = plt.figure(figsize=(8, 6), dpi=300)

    G, role_id, plugins = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.01)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_cycle"


    return G, role_id, name


def treegrid(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["grid", 6]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_grid"


    return G, role_id, name
def treehouse(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["house"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_house"


    return G, role_id, name
def treefan(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["fan"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_fan"


    return G, role_id, name
def treeclique(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["clique"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_clique"


    return G, role_id, name

def treediamond(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["diamond"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_diamond"


    return G, role_id, name

def treestar(nb_shapes=100, width_basis=4, feature_generator=None, m=4):
    """ Synthetic Graph #5:
    Start with a tree and attach grid-shaped subgraphs.
    Args:
        nb_shapes         :  The number of shapes (here 'houses') that should be added to the base graph.
        width_basis       :  The width of the basis graph (here a random 'grid').
        feature_generator :  A `FeatureGenerator` for node features. If `None`, add constant features to nodes.
        m                 :  The tree depth.
    Returns:
        G                 :  A networkx graph
        role_id           :  Role ID for each node in synthetic graph
        name              :  A graph identifier
    """
    basis_type = "tree"
    list_shapes = [["star"]] * nb_shapes


    G, role_id, _ = build_graph(
        width_basis, basis_type, list_shapes, start=0
    )
    G = perturb([G], 0.1)[0]

    if feature_generator is None:
        feature_generator = ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)  + "_star"


    return G, role_id, name


In [ ]:
feature = get_from_json("../Dataset/features.json")
def syn_motif_ba(feature):
    for function in [bahouse,bafan,baclique,badiamond,bacycle,bastar,bagrid]:
        for j in range(1,11):
            for i in range(1,6):
                G_conf,_ ,name = function(nb_shapes=10*i,width_basis=200)
                feat = global_feature(G_conf)
                #plt.figure(figsize=(8, 6), dpi=300)
                #nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 50, with_labels=False)
                features.append(feat)
                name = name+'_'+str(j)
                file_name =  "../Dataset/Motifs_Attached/New_BA/"+name
                nx.write_edgelist(G_conf,file_name+str(".txt"))
                feature[name] = global_feature(G_conf)
    return feature
feature = syn_motif_ba(feature)
with open("../Dataset/features.json", "w") as outfile:
    json.dump(feature, outfile, cls=NpEncoder)

In [ ]:
feature = get_from_json("../Dataset/features.json")
def syn_motif_tree(feature):
    for function in [treehouse,treefan,treeclique,treediamond,treecycle,treestar,treegrid]:
        for j in range(1,11):
            for i in range(1,6):
                G_conf,_ ,name = function(nb_shapes=10*i,width_basis=5)
                feat = global_feature(G_conf)
                #plt.figure(figsize=(8, 6), dpi=300)
                #nx.draw(G_conf,nodelist=G_conf.nodes, font_color='white' ,node_size = 50, with_labels=False)
                name = name+'_'+str(j)
                file_name =  "../Dataset/Motifs_Attached/New_Tree/"+name
                nx.write_edgelist(G_conf,file_name+str(".txt"))
                feature[name] = global_feature(G_conf)
    return feature
feature = syn_motif_tree(feature)
with open("../Dataset/features.json", "w") as outfile:
    json.dump(feature, outfile, cls=NpEncoder)
    

# Onion Model

In [61]:
import networkx as nx
import random
import matplotlib.pyplot as plt
def plotFigure(new, old):
    plt.plot(new, label="new")
    plt.plot(old, label="old")
    plt.ylabel("Largest Connected Components")
    plt.xlabel("Number of Nodes Removed")
    plt.legend()
    plt.show()
def RankednodeRemoval(G):
    list_lcc = [len(max(nx.connected_components(G), key=len))]
    list_Nodes = list(G.nodes())
    for i in list_Nodes:
        degreeCentrality = nx.degree_centrality(G)
        highestRanked = sorted( degreeCentrality, key=lambda x: degreeCentrality[x], reverse=True)[0]
        G.remove_node(highestRanked)
        lcc = len(max(nx.connected_components(G), key=len))
        list_lcc.append(lcc)
        if lcc == 1:
            return(list_lcc)
def RandomnodeRemoval(G):
    list_lcc = [len(max(nx.connected_components(G), key=len))]
    removedNodes = [] 
    list_Nodes = list(G.nodes())
    for i in list_Nodes:
        nodes= list(G.nodes())
        selectedNode = random.choice(nodes)
        G.remove_node(selectedNode)
        lcc = len(max(nx.connected_components(G), key=len))
        list_lcc.append(lcc)
        if lcc == 1:
            return(list_lcc)
def onion_swap(G):
    """
    Swaps two randomly chosen edges in the graph G and checks if the robustness of the
    network is improved. If the swap is accepted, the new graph is returned. If the swap is
    rejected, the original graph is returned.
    """
    N = len(G)
    # Choose two random edges to swap
    edges = list(G.edges())
    e1, e2 = random.sample(edges, 2)
    i, j = e1
    k, l = e2
    # Create the new edges
    e3 = (i, k)
    e4 = (j, l)
    # Calculate the robustness of the original and new graphs
    Rold = 1/(N+1)*sum(RankednodeRemoval(G.copy()))#len(max(nx.connected_components(G), key=len))
    Gnew = G.copy()
    Gnew.remove_edges_from([e1, e2])
    Gnew.add_edges_from([e3, e4])
    Rnew = 1/(N+1)*sum(RankednodeRemoval(Gnew.copy())) #len(max(nx.connected_components(Gnew), key=len))
    # If the robustness is improved, accept the swap and return the new graph
    if Rnew > Rold:
        return Gnew
    
    # Otherwise, reject the swap and return the original graph
    return G
def get_OnionModel(feature,node_size, scale, threshold_iteration):
    for n in node_size:
        for i in range(1,6):
            cond = True
            while(cond):
                degree_sequence = nx.utils.powerlaw_sequence(n, scale)
                degree_sequence = [int(d) for d in degree_sequence]
                if (sum(degree_sequence)%2==0):
                    break 
            G = nx.configuration_model(degree_sequence)
            lccOld = RankednodeRemoval(G.copy())
            for a in range(threshold_iteration):
                G = onion_swap(G)
            G = nx.Graph(G)
            G.remove_edges_from(nx.selfloop_edges(G))
            lccNew = RankednodeRemoval(G.copy())
            name = "onion_"+str(n)+"_"+str(scale)+"_"+str(i)
            file_name =  "../Dataset/SyntheticGraph/OnionModel/"+name
            nx.write_edgelist(G,file_name+".txt")
            feature["synthetic"][name] = global_feature(G)
            #plotFigure(lccNew,lccOld)
            print(file_name)
    return feature


In [64]:
numNodes = [100,200,300,400,500]
features = get_from_json("../Dataset/featuresNew.json")
newfeatures= get_OnionModel(features,numNodes, scale=2.5, threshold_iteration=500)
print(newfeatures)

../Dataset/SyntheticGraph/OnionModel/onion_100_2.5_1
../Dataset/SyntheticGraph/OnionModel/onion_100_2.5_2
../Dataset/SyntheticGraph/OnionModel/onion_100_2.5_3
../Dataset/SyntheticGraph/OnionModel/onion_100_2.5_4
../Dataset/SyntheticGraph/OnionModel/onion_100_2.5_5
../Dataset/SyntheticGraph/OnionModel/onion_200_2.5_1
../Dataset/SyntheticGraph/OnionModel/onion_200_2.5_2
../Dataset/SyntheticGraph/OnionModel/onion_200_2.5_3
../Dataset/SyntheticGraph/OnionModel/onion_200_2.5_4
../Dataset/SyntheticGraph/OnionModel/onion_200_2.5_5
../Dataset/SyntheticGraph/OnionModel/onion_300_2.5_1
../Dataset/SyntheticGraph/OnionModel/onion_300_2.5_2
../Dataset/SyntheticGraph/OnionModel/onion_300_2.5_3
../Dataset/SyntheticGraph/OnionModel/onion_300_2.5_4
../Dataset/SyntheticGraph/OnionModel/onion_300_2.5_5
../Dataset/SyntheticGraph/OnionModel/onion_400_2.5_1
../Dataset/SyntheticGraph/OnionModel/onion_400_2.5_2
../Dataset/SyntheticGraph/OnionModel/onion_400_2.5_3
../Dataset/SyntheticGraph/OnionModel/onion_400

In [65]:
with open("../Dataset/featuresNew.json", "w") as outfile:
    json.dump(newfeatures, outfile, cls=NpEncoder)